In [33]:
import pandas as pd
import numpy as np
import math
from math import ceil
# from uncertainties import ufloat

In [19]:
df = pd.DataFrame({'N': [1,2,3,1,2,3,1,2,3], 'Z': [1,1,1,2,2,2,3,3,3], 'BE': [1,2,3,4,5,6,7,8,9], 'BE_W1': [1,2,3,4,5,6,7,8,9]})
df = pd.DataFrame({'N': [2,2,2,2,2], 'Z': [2,3,4,5,6], 'BE': [1,2,3,4,5], 'BE_W1': [1,2,3,4,5]})
df = pd.DataFrame({'N': [2,2,2], 'Z': [2,4,6], 'BE': [1,3,5], 'BE_W1': [1,3,5]})
df['BE']/(df['N']+df['Z'])
# TwoPSGap(df,0)
# (-1)**(df['Z'])



0    0.250
1    0.500
2    0.625
dtype: float64

In [59]:

Wstring = {0: '', 1: '_W1', 2: '_W2'}

def OneNSE(df, W):
    shifted_df = df[['N','Z','BE'+Wstring[W]]].copy()
    shifted_df['N'] = shifted_df['N'].add(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['OneNSE'+Wstring[W]] = df['BE'+Wstring[W]] - df['BE'+Wstring[W]+'p']
    # df['OneNSE'+Wstring[W]] = df['BE'+Wstring[W]] - df.groupby(['Z'])['BE'+Wstring[W]].shift(1)
    return df

def OnePSE(df, W):
    shifted_df = df[['N','Z','BE'+Wstring[W]]].copy()
    shifted_df['Z'] = shifted_df['Z'].add(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['OnePSE'+Wstring[W]] = df['BE'+Wstring[W]] - df['BE'+Wstring[W]+'p']
    # df['OnePSE'+Wstring[W]] = df['BE'+Wstring[W]] - df.groupby(['N'])['BE'+Wstring[W]].shift(1)
    return df

def TwoNSE(df, W):
    shifted_df = df[['N','Z','BE'+Wstring[W]]].copy()
    shifted_df['N'] = shifted_df['N'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['TwoNSE'+Wstring[W]] = df['BE'+Wstring[W]] - df['BE'+Wstring[W]+'p']
    # df['TwoNSE'+Wstring[W]] = df['BE'+Wstring[W]] - df.groupby(['Z'])['BE'+Wstring[W]].shift(2)
    return df

def TwoPSE(df, W):
    shifted_df = df[['N','Z','BE'+Wstring[W]]].copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['TwoPSE'+Wstring[W]] = df['BE'+Wstring[W]] - df['BE'+Wstring[W]+'p']
    # df['TwoPSE'+Wstring[W]] = df['BE'+Wstring[W]] - df.groupby(['N'])['BE'+Wstring[W]].shift(2)
    return df

def AlphaSE(df, W):
    shifted_df = df[['N','Z','BE'+Wstring[W]]].copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    shifted_df['N'] = shifted_df['N'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['AlphaSE'+Wstring[W]] = df['BE'+Wstring[W]] - df['BE'+Wstring[W]+'p'] - 28.3
    return df

def TwoNSGap(df, W):
    try:
        df = df[['N','Z','TwoNSE'+Wstring[W]]]
    except:
        df = TwoNSE(df,W)[['N','Z','TwoNSE'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['TwoNSGap'+Wstring[W]] = df['TwoNSE'+Wstring[W]] - df['TwoNSE'+Wstring[W]+'p']
    return df

def TwoPSGap(df, W):
    try:
        df = df[['N','Z','TwoPSE'+Wstring[W]]]
    except:
        df = TwoPSE(df,W)[['N','Z','TwoPSE'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['TwoPSGap'+Wstring[W]] = df['TwoPSE'+Wstring[W]] - df['TwoPSE'+Wstring[W]+'p']
    # df['TwoPSGap'+Wstring[W]] = TwoPSE(df, W)['TwoPSE'+Wstring[W]] - TwoPSE(df, W).groupby(['N'])['TwoPSE'+Wstring[W]].shift(-2)
    return df

def DoubleMDiff(df, W):
    try:
        df = df[['N','Z','TwoPSE'+Wstring[W]]]
    except:
        df = TwoPSE(df,W)[['N','Z','TwoPSE'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['DoubleMDiff'+Wstring[W]] = (df['TwoPSE'+Wstring[W]] - df['TwoPSE'+Wstring[W]+'p'])/4
    return df
    # return (TwoPSE(Z,N,W,df) - TwoPSE(Z,N-2,W,df))/4

def N3PointOED(df, W):
    try:
        df = df[['N','Z','OneNSE'+Wstring[W]]]
    except:
        df = OneNSE(df,W)[['N','Z','OneNSE'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['N3PointOED'+Wstring[W]] = (-1)**df['N']*(df['OneNSE'+Wstring[W]] - df['OneNSE'+Wstring[W]+'p'])/2
    return df
    # return (-1)**N*(OneNSE(Z,N,W,df) - OneNSE(Z,N+1,W,df))/2

def P3PointOED(df, W):
    try:
        df = df[['N','Z','OnePSE'+Wstring[W]]]
    except:
        df = OnePSE(df,W)[['N','Z','OnePSE'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['P3PointOED'+Wstring[W]] = (-1)**df['Z']*(df['OnePSE'+Wstring[W]] - df['OnePSE'+Wstring[W]+'p'])/2
    return df
    # return (-1)**Z*(OnePSE(Z,N,W,df) - OnePSE(Z+1,N,W,df))/2

def SNESplitting(df, W):
    try:
        df = df[['N','Z','OneNSE'+Wstring[W]]]
    except:
        df = OneNSE(df,W)[['N','Z','OneNSE'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['SNESplitting'+Wstring[W]] = (-1)**df['N']*(df['OneNSE'+Wstring[W]] - df['OneNSE'+Wstring[W]+'p'])
    return df
    # return (-1)**N*(OneNSE(Z,N,W,df) - OneNSE(Z,N+2,W,df))

def SPESplitting(df, W):
    try:
        df = df[['N','Z','OnePSE'+Wstring[W]]]
    except:
        df = OnePSE(df,W)[['N','Z','OnePSE'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['SPESplitting'+Wstring[W]] = (-1)**df['Z']*(df['OnePSE'+Wstring[W]] - df['OnePSE'+Wstring[W]+'p'])
    return df
    # return (-1)**Z*(OnePSE(Z,N,W,df) - OnePSE(Z+2,N,W,df))

def WignerEC(df, W):
    try:
        df = df[['N','Z','DoubleMDiff'+Wstring[W]]]
    except:
        df = DoubleMDiff(df,W)[['N','Z','DoubleMDiff'+Wstring[W]]]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    shifted_df2 = df.copy()
    shifted_df2['N'] = shifted_df2['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df = pd.merge(df, shifted_df2, on=['Z', 'N'], how='left', suffixes=('', 'p2'))
    df['WignerEC'+Wstring[W]] = df['DoubleMDiff'+Wstring[W]]-(df['DoubleMDiff'+Wstring[W]+'p'] + df['DoubleMDiff'+Wstring[W]+'p2'])/2
    df['WignerEC'+Wstring[W]] = np.where(df['N']==df['Z'], df['WignerEC'+Wstring[W]], np.nan)
    return df

def BEperA(df, W):
    df['BEperA'+Wstring[W]] = df['BE'+Wstring[W]]/(df['Z']+df['N'])
    return df


def uOneNSE(df):
    shifted_df = df[['N','Z','uBE']].copy()
    shifted_df['N'] = shifted_df['N'].add(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uOneNSE'] = df['uBE'] + df['uBEp']
    return df

def uOnePSE(df):
    shifted_df = df[['N','Z','uBE']].copy()
    shifted_df['Z'] = shifted_df['Z'].add(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uOnePSE'] = df['uBE'] + df['uBEp']
    return df

def uTwoNSE(df):
    shifted_df = df[['N','Z','uBE']].copy()
    shifted_df['N'] = shifted_df['N'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uTwoNSE'] = df['uBE'] + df['uBEp']
    return df

def uTwoPSE(df):
    shifted_df = df[['N','Z','uBE']].copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uTwoPSE'] = df['uBE'] + df['uBEp']
    return df

def uAlphaSE(df):
    shifted_df = df[['N','Z','uBE']].copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    shifted_df['N'] = shifted_df['N'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uAlphaSE'] = df['uBE'] + df['uBEp']
    return df

def uTwoNSGap(df):
    try:
        df = df[['N','Z','uTwoNSE']]
    except:
        df = uTwoNSE(df)[['N','Z','uTwoNSE']]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uTwoNSGap'] = df['uTwoNSE'] + df['uTwoNSEp']
    return df

def uTwoPSGap(df):
    try:
        df = df[['N','Z','uTwoPSE']]
    except:
        df = uTwoPSE(df)[['N','Z','uTwoPSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uTwoPSGap'] = df['uTwoPSE'] - df['uTwoPSEp']
    return df

def uDoubleMDiff(df):
    try:
        df = df[['N','Z','uTwoPSE']]
    except:
        df = uTwoPSE(df)[['N','Z','uTwoPSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uDoubleMDiff'] = (df['uTwoPSE'] + df['uTwoPSEp'])/2
    return df

def uN3PointOED(df):
    try:
        df = df[['N','Z','uOneNSE']]
    except:
        df = uOneNSE(df)[['N','Z','uOneNSE']]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uN3PointOED'] = (df['uOneNSE'] + df['uOneNSEp'])/2
    return df

def uP3PointOED(df):
    try:
        df = df[['N','Z','uOnePSE']]
    except:
        df = uOnePSE(df)[['N','Z','uOnePSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uP3PointOED'] = (df['uOnePSE'] + df['uOnePSEp'])/2
    return df

def uSNESplitting(df):
    try:
        df = df[['N','Z','uOneNSE']]
    except:
        df = uOneNSE(df)[['N','Z','uOneNSE']]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uSNESplitting'] = df['uOneNSE'] + df['uOneNSEp']
    return df

def uSPESplitting(df):
    try:
        df = df[['N','Z','uOnePSE']]
    except:
        df = uOnePSE(df)[['N','Z','uOnePSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['uSPESplitting'] = df['uOnePSE'] + df['uOnePSEp']
    return df

def uWignerEC(df):
    try:
        df = df[['N','Z','uDoubleMDiff']]
    except:
        df = uDoubleMDiff(df)[['N','Z','uDoubleMDiff']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    shifted_df2 = df.copy()
    shifted_df2['N'] = shifted_df2['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df = pd.merge(df, shifted_df2, on=['Z', 'N'], how='left', suffixes=('', 'p2'))
    df['uWignerEC'] = df['uDoubleMDiff'] + (df['uDoubleMDiffp'] + df['uDoubleMDiffp2'])/2
    df['uWignerEC'] = np.where(df['N']==df['Z'], df['uWignerEC'], np.nan)
    return df

def uBEperA(df):
    df['uBEperA'] = df['uBE']/(df['Z']+df['N'])
    return df


def eOneNSE(df):
    shifted_df = df[['N','Z','eBE']].copy()
    shifted_df['N'] = shifted_df['N'].add(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eOneNSE'] = ((df['eBE'] + df['eBEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eOnePSE(df):
    shifted_df = df[['N','Z','eBE']].copy()
    shifted_df['Z'] = shifted_df['Z'].add(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eOnePSE'] = ((df['eBE'] + df['eBEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eTwoNSE(df):
    shifted_df = df[['N','Z','eBE']].copy()
    shifted_df['N'] = shifted_df['N'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eTwoNSE'] = ((df['eBE'] + df['eBEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eTwoPSE(df):
    shifted_df = df[['N','Z','eBE']].copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eTwoPSE'] = ((df['eBE'] + df['eBEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eAlphaSE(df):
    shifted_df = df[['N','Z','eBE']].copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    shifted_df['N'] = shifted_df['N'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eAlphaSE'] = ((df['eBE'] + df['eBEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eTwoNSGap(df):
    try:
        df = df[['N','Z','eTwoNSE']]
    except:
        df = eTwoNSE(df)[['N','Z','eTwoNSE']]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eTwoNSGap'] = ((df['eTwoNSE'] + df['eTwoNSEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eTwoPSGap(df):
    try:
        df = df[['N','Z','eTwoPSE']]
    except:
        df = eTwoPSE(df)[['N','Z','eTwoPSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eTwoPSGap'] = ((df['eTwoPSE'] - df['eTwoPSEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eDoubleMDiff(df):
    try:
        df = df[['N','Z','eTwoPSE']]
    except:
        df = eTwoPSE(df)[['N','Z','eTwoPSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eDoubleMDiff'] = ((df['eTwoPSE'] + df['eTwoPSEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eN3PointOED(df):
    try:
        df = df[['N','Z','eOneNSE']]
    except:
        df = eOneNSE(df)[['N','Z','eOneNSE']]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eN3PointOED'] = ((df['eOneNSE'] + df['eOneNSEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eP3PointOED(df):
    try:
        df = df[['N','Z','eOnePSE']]
    except:
        df = eOnePSE(df)[['N','Z','eOnePSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(1)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eP3PointOED'] = ((df['eOnePSE'] + df['eOnePSEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eSNESplitting(df):
    try:
        df = df[['N','Z','eOneNSE']]
    except:
        df = eOneNSE(df)[['N','Z','eOneNSE']]
    shifted_df = df.copy()
    shifted_df['N'] = shifted_df['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eSNESplitting'] = ((df['eOneNSE'] + df['eOneNSEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eSPESplitting(df):
    try:
        df = df[['N','Z','eOnePSE']]
    except:
        df = eOnePSE(df)[['N','Z','eOnePSE']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df['eSPESplitting'] = ((df['eOnePSE'] + df['eOnePSEp'])/2).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    return df

def eWignerEC(df):
    try:
        df = df[['N','Z','eDoubleMDiff']]
    except:
        df = eDoubleMDiff(df)[['N','Z','eDoubleMDiff']]
    shifted_df = df.copy()
    shifted_df['Z'] = shifted_df['Z'].add(2)
    shifted_df2 = df.copy()
    shifted_df2['N'] = shifted_df2['N'].sub(2)
    df = pd.merge(df, shifted_df, on=['Z', 'N'], how='left', suffixes=('', 'p'))
    df = pd.merge(df, shifted_df2, on=['Z', 'N'], how='left', suffixes=('', 'p2'))
    df['eWignerEC'] = ((df['eDoubleMDiff']+df['eDoubleMDiffp'] + df['eDoubleMDiffp2'])/3).apply(lambda x: ceil(x) if not np.isnan(x) else 2)
    df['eWignerEC'] = np.where(df['N']==df['Z'], df['eWignerEC'], np.nan)
    return df

def eBEperA(df):
    df['eBEperA'] = df['eBE']
    return df

    
def Wig1(Z, N):
    return (1.8*np.exp(-380*((N-Z)/(N+Z))**2))-(.84*abs(N-Z)*np.exp(-(((N+Z)/26)**2)))

def Wig2(Z, N):
    return -47*(abs(N-Z)/(N+Z))

def Identity(Z, N):
    return 0

In [2]:
import pandas as pd
x = pd.read_hdf('7-3-23.h5', key='HFB24')
x.columns

Index(['Z', 'N', 'BE', 'bet2', 'bet4', 'Rch', 'Edef', 'Sn', 'Sp', 'Qbet',
       'Mcal', 'Mexp-Mcal', 'Jexp', 'Jth', 'Pexp', 'Pth', 'BE_W1', 'BE_W2',
       'BEperA', 'BEperA_W1', 'BEperA_W2', 'OneNSE', 'OnePSE', 'TwoNSE',
       'TwoPSE', 'AlphaSE', 'TwoNSGap', 'TwoPSGap', 'DoubleMDiff',
       'N3PointOED', 'P3PointOED', 'SNESplitting', 'SPESplitting', 'WignerEC',
       'OneNSE_W1', 'OnePSE_W1', 'TwoNSE_W1', 'TwoPSE_W1', 'AlphaSE_W1',
       'TwoNSGap_W1', 'TwoPSGap_W1', 'DoubleMDiff_W1', 'N3PointOED_W1',
       'P3PointOED_W1', 'SNESplitting_W1', 'SPESplitting_W1', 'WignerEC_W1',
       'OneNSE_W2', 'OnePSE_W2', 'TwoNSE_W2', 'TwoPSE_W2', 'AlphaSE_W2',
       'TwoNSGap_W2', 'TwoPSGap_W2', 'DoubleMDiff_W2', 'N3PointOED_W2',
       'P3PointOED_W2', 'SNESplitting_W2', 'SPESplitting_W2', 'WignerEC_W2'],
      dtype='object')

In [61]:
import pandas as pd
x = pd.read_hdf('7-3-23.h5', 'AME2020')
x = x[x['Z']==40]
x[['eSNESplitting','eBE']]


,eSNESplitting,eBE
1010,2,1
1011,1,1
1012,1,1
1013,1,1
1014,1,0
1015,0,0
1016,0,0
1017,0,0
1018,0,0
1019,0,0


In [60]:

sheet_names = ['AME2020', 'DD-ME2', 'DD-MEdelta', 'DD-PC1', 'NL3S', 'SKMS', 'SKP', 'SLY4', 'SV', \
               'UNEDF0', 'UNEDF1', 'UNEDF2', 'FRDM12', 'HFB24', 'BCPM', 'D1M']
mnames = ['AME2020', 'ME2', 'MEdelta', 'PC1', 'NL3S', 'SKMS', 'SKP', 'SLY4', 'SV', 'UNEDF0', \
        'UNEDF1', 'UNEDF2', 'FRDM12', 'HFB24', 'BCPM', 'D1M']
qfuncs = [OneNSE, OnePSE, TwoNSE, TwoPSE, AlphaSE, TwoNSGap, TwoPSGap, DoubleMDiff, N3PointOED, P3PointOED, 
        SNESplitting, SPESplitting, WignerEC, BEperA] 
exp_qfuncs = [uOneNSE, uOnePSE, uTwoNSE, uTwoPSE, uAlphaSE, uTwoNSGap, 
        uTwoPSGap, uDoubleMDiff, uN3PointOED, uP3PointOED, uSNESplitting, uSPESplitting, uWignerEC, uBEperA,
        eOneNSE, eOnePSE, eTwoNSE, eTwoPSE, eAlphaSE, eTwoNSGap, eTwoPSGap, eDoubleMDiff, 
        eN3PointOED, eP3PointOED, eSNESplitting, eSPESplitting, eWignerEC, eBEperA]
for model_index in range(len(sheet_names)):
    model = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = sheet_names[model_index])
    model = model.sort_values(['Z', 'N'])
    model.dropna(how='all', inplace=True)
    model['BE_W1'] = model['BE'] - Wig1(model['Z'], model['N'])
    model['BE_W2'] = model['BE'] - Wig2(model['Z'], model['N'])
    qnames = ['OneNSE', 'OnePSE', 'TwoNSE', 'TwoPSE', 'AlphaSE', 'TwoNSGap', 'TwoPSGap', 'DoubleMDiff', 'N3PointOED', 
            'P3PointOED', 'SNESplitting', 'SPESplitting', 'WignerEC', 'BEperA', 'OneNSE_W1', 'OnePSE_W1', 'TwoNSE_W1', 
            'TwoPSE_W1', 'AlphaSE_W1', 'TwoNSGap_W1', 'TwoPSGap_W1', 'DoubleMDiff_W1', 'N3PointOED_W1', 'P3PointOED_W1', 
            'SNESplitting_W1', 'SPESplitting_W1', 'WignerEC_W1', 'BEperA_W1','OneNSE_W2', 'OnePSE_W2', 'TwoNSE_W2', 
            'TwoPSE_W2', 'AlphaSE_W2', 'TwoNSGap_W2', 'TwoPSGap_W2', 'DoubleMDiff_W2', 'N3PointOED_W2', 'P3PointOED_W2', 
            'SNESplitting_W2', 'SPESplitting_W2', 'WignerEC_W2', 'BEperA_W2']
    if model_index == 0:
        qnames += ['uOneNSE', 'uOnePSE', 'uTwoNSE', 'uTwoPSE', 'uAlphaSE', 'uTwoNSGap', 'uTwoPSGap', 'uDoubleMDiff', 
                   'uN3PointOED', 'uP3PointOED', 'uSNESplitting', 'uSPESplitting', 'uWignerEC', 'uBEperA', 'eOneNSE', 
                   'eOnePSE', 'eTwoNSE', 'eTwoPSE', 'eAlphaSE', 'eTwoNSGap', 'eTwoPSGap', 'eDoubleMDiff', 
                   'eN3PointOED', 'eP3PointOED', 'eSNESplitting', 'eSPESplitting', 'eWignerEC', 'eBEperA']
    quan_df = pd.DataFrame(columns=qnames, index=range(len(model.index)))
    for W in range(3):
        for q in qfuncs:
                quan_df[q.__name__+Wstring[W]] = q(model, W)[q.__name__+Wstring[W]]
    if model_index == 0:
        for q in exp_qfuncs:
            quan_df[q.__name__] = q(model)[q.__name__]
    master_df = pd.concat([model, quan_df], axis=1)
    duplicate_columns = master_df.columns.duplicated()
    duplicate_column_names = master_df.columns[duplicate_columns] # Extract the names of the duplicate columns
    master_df = master_df.loc[:, ~duplicate_columns]
    master_df.to_hdf('7-3-23.h5', key= mnames[model_index], index=False)
    print(sheet_names[model_index], 'done')

AME2020 done
DD-ME2 done
DD-MEdelta done
DD-PC1 done
NL3S done
SKMS done
SKP done
SLY4 done
SV done
UNEDF0 done
UNEDF1 done
UNEDF2 done
FRDM12 done
HFB24 done
BCPM done
D1M done


In [16]:
x = pd.read_hdf('6-30-23.h5', key='AME2020')
x.columns

Index(['BE', 'uBE', 'N', 'Z', 'eBE', 'BE_W1', 'BE_W2', 'OneNSE', 'OnePSE',
       'TwoNSE', 'TwoPSE', 'AlphaSE', 'TwoNSGap', 'TwoPSGap', 'DoubleMDiff',
       'N3PointOED', 'P3PointOED', 'SNESplitting', 'SPESplitting', 'WignerEC',
       'OneNSE_W1', 'OnePSE_W1', 'TwoNSE_W1', 'TwoPSE_W1', 'AlphaSE_W1',
       'TwoNSGap_W1', 'TwoPSGap_W1', 'DoubleMDiff_W1', 'N3PointOED_W1',
       'P3PointOED_W1', 'SNESplitting_W1', 'SPESplitting_W1', 'WignerEC_W1',
       'OneNSE_W2', 'OnePSE_W2', 'TwoNSE_W2', 'TwoPSE_W2', 'AlphaSE_W2',
       'TwoNSGap_W2', 'TwoPSGap_W2', 'DoubleMDiff_W2', 'N3PointOED_W2',
       'P3PointOED_W2', 'SNESplitting_W2', 'SPESplitting_W2', 'WignerEC_W2',
       'uOneNSE', 'uOnePSE', 'uTwoNSE', 'uTwoPSE', 'uAlphaSE', 'uTwoNSGap',
       'uTwoPSGap', 'uDoubleMDiff', 'uN3PointOED', 'uP3PointOED',
       'uSNESplitting', 'uSPESplitting', 'uWignerEC', 'uBEperA', 'eOneNSE',
       'eOnePSE', 'eTwoNSE', 'eTwoPSE', 'eAlphaSE', 'eTwoNSGap', 'eTwoPSGap',
       'eDoubleMDiff', 'eN3Po

In [58]:
master_df

,BE,uBE,N,Z,eBE,BE_W1,BE_W2,OneNSE,OnePSE,TwoNSE,...,eAlphaSE,eTwoNSGap,eTwoPSGap,eDoubleMDiff,eN3PointOED,eP3PointOED,eSNESplitting,eSPESplitting,eWignerEC,eBEperA
0,2.224566,4.000000e-07,1,1,0,0.424566,2.224566,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
1,8.481796,9.000000e-07,2,1,0,9.310687,24.148463,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
2,6.881796,1.000000e-01,3,1,0,8.522500,30.381796,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
3,6.681796,8.944250e-02,4,1,0,9.110303,34.881796,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
4,5.769837,2.541270e-01,5,1,0,8.955583,37.103170,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3551,2078.835000,8.790000e-01,176,117,1,2078.835000,2088.299164,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,1
3552,2085.048000,5.880000e-01,177,117,1,2085.048000,2094.639837,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,1
3553,2073.854000,5.860000e-01,175,118,1,2073.853999,2082.997345,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,1
3554,2081.226000,5.880000e-01,176,118,1,2081.225999,2090.498109,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,1


In [ ]:

Wstring = {0: '', 1: '_W1', 2: '_W2'}

def BE(Z,N,W,df):
    
    return df[(df['Z']==Z) & (df['N']==N)]['BE'+Wstring[W]]

def OneNSE(Z,N,W,df):
    return BE(Z,N,W,df) - BE(Z,N-1,W,df)

def OnePSE(Z,N,W,df):
    return BE(Z,N,W,df) - BE(Z-1,N,W,df)

def TwoNSE(Z,N,W,df):
    return BE(Z,N,W,df) - BE(Z,N-2,W,df)

def TwoPSE(Z,N,W,df):
    return BE(Z,N,W,df) - BE(Z-2,N,W,df)

def AlphaSE(Z,N,W,df):
    return BE(Z,N,W,df) - BE(Z-2,N-2,W,df) - 28.3

def TwoPSGap(Z,N,W,df):
    return TwoPSE(Z,N,W,df) - TwoPSE(Z+2,N,W,df)

def TwoNSGap(Z,N,W,df):
    return TwoNSE(Z,N,W,df) - TwoNSE(Z,N+2,W,df)

def DoubleMDiff(Z,N,W,df):
    return (TwoPSE(Z,N,W,df) - TwoPSE(Z,N-2,W,df))/4

def N3PointOED(Z,N,W,df):
    return (-1)**N*(OneNSE(Z,N,W,df) - OneNSE(Z,N+1,W,df))/2

def P3PointOED(Z,N,W,df):
    return (-1)**Z*(OnePSE(Z,N,W,df) - OnePSE(Z+1,N,W,df))/2

def SNESplitting(Z,N,W,df):
    return (-1)**N*(OneNSE(Z,N,W,df) - OneNSE(Z,N+2,W,df))

def SPESplitting(Z,N,W,df):
    return (-1)**Z*(OnePSE(Z,N,W,df) - OnePSE(Z+2,N,W,df))

def WignerEC(Z,N,W,df):
    if isinstance(Z, (np.ndarray, pd.Series)):
        WECarr = DoubleMDiff(Z, N, W, df) - (DoubleMDiff(Z - 2, N, W, df) + DoubleMDiff(Z, N + 2, W, df))/2
        return np.where(Z == N, WECarr, np.nan)
    elif Z!=N:
        return None
    return DoubleMDiff(Z,N,W,df)-(DoubleMDiff(Z-2,N,W,df) + DoubleMDiff(Z,N+2,W,df))/2

def BEperA(Z,N,W,df):
    return BE(Z,N,W,df)/(Z+N)
    
def uBE(Z,N,W,df):
    try:
        return df[(df['Z']==Z) & (df['N']==N)]['uBE']
    except:
        return None

def uOneNSE(Z,N,W,df):
    try:
        return uBE(Z,N,W,df) + uBE(Z,N-1,W,df)
    except:
        return None
    
def uOnePSE(Z,N,W,df):
    try:
        return uBE(Z,N,W,df) + uBE(Z-1,N,W,df)
    except:
        return None

def uTwoNSE(Z,N,W,df):
    try:
        return uBE(Z,N,W,df) + uBE(Z,N-2,W,df)
    except:
        return None

def uTwoPSE(Z,N,W,df):
    try:
        return uBE(Z,N,W,df) + uBE(Z-2,N,W,df)
    except:
        return None

def uAlphaSE(Z,N,W,df):
    try:
        return uBE(Z,N,W,df) + uBE(Z-2,N-2,W,df)
    except:
        return None

def uTwoPSGap(Z,N,W,df):
    try:
        return uTwoPSE(Z,N,W,df) + uTwoPSE(Z+2,N,W,df)
    except:
        return None

def uTwoNSGap(Z,N,W,df):
    try:
        return uTwoNSE(Z,N,W,df) + uTwoNSE(Z,N+2,W,df)
    except:
        return None

def uDoubleMDiff(Z,N,W,df):
    try:
        return (uTwoPSE(Z,N,W,df) + uTwoPSE(Z,N-2,W,df))/4
    except:
        return None

def uN3PointOED(Z,N,W,df):
    try:
        return (uOneNSE(Z,N,W,df) + uOneNSE(Z,N+1,W,df))/2
    except:
        return None

def uP3PointOED(Z,N,W,df):
    try:
        return (uOnePSE(Z,N,W,df) + uOnePSE(Z+1,N,W,df))/2
    except:
        return None

def uSNESplitting(Z,N,W,df):
    try:
        return (uOneNSE(Z,N,W,df) + uOneNSE(Z,N+2,W,df))
    except:
        return None

def uSPESplitting(Z,N,W,df):
    try:
        return (uOnePSE(Z,N,W,df) + uOnePSE(Z+2,N,W,df))
    except:
        return None

def uWignerEC(Z,N,W,df):
    if isinstance(Z, (np.ndarray, pd.Series)):
        WECarr = uDoubleMDiff(Z,N,W,df)+(uDoubleMDiff(Z-2,N,W,df) + uDoubleMDiff(Z,N+2,W,df))/2
        return np.where(Z == N, WECarr, np.nan)
    elif Z!=N:
        return None
    return uDoubleMDiff(Z,N,W,df)+(uDoubleMDiff(Z-2,N,W,df) + uDoubleMDiff(Z,N+2,W,df))/2

def uBEperA(Z,N,W,df):
    try:
        return uBE(Z,N,W,df)/(Z+N)
    except:
        return None
    
def eBE(Z,N,W,df):
    try:
        return df[(df['Z']==Z) & (df['N']==N)]['eBE']
    except:
        return np.nan

def eOneNSE(Z,N,W,df):
    try:
        return ceil((eBE(Z,N,W,df) + eBE(Z,N-1,W,df))/2)
    except:
        return np.nan
    
def eOnePSE(Z,N,W,df):
    try:
        return ceil((eBE(Z,N,W,df) + eBE(Z-1,N,W,df))/2)
    except:
        return np.nan

def eTwoNSE(Z,N,W,df):
    try:
        return ceil((eBE(Z,N,W,df) + eBE(Z,N-2,W,df))/2)
    except:
        return np.nan

def eTwoPSE(Z,N,W,df):
    try:
        return ceil((eBE(Z,N,W,df) + eBE(Z-2,N,W,df))/2)
    except:
        return np.nan

def eAlphaSE(Z,N,W,df):
    try:
        return ceil((eBE(Z,N,W,df) + eBE(Z-2,N-2,W,df))/2)
    except:
        return np.nan

def eTwoPSGap(Z,N,W,df):
    try:
        return ceil((eTwoPSE(Z,N,W,df) + eTwoPSE(Z+2,N,W,df))/2)
    except:
        return np.nan

def eTwoNSGap(Z,N,W,df):
    try:
        return ceil((eTwoNSE(Z,N,W,df) + eTwoNSE(Z,N+2,W,df))/2)
    except:
        return np.nan

def eDoubleMDiff(Z,N,W,df):
    try:
        return ceil((eTwoPSE(Z,N,W,df) + eTwoPSE(Z,N-2,W,df))/2)
    except:
        return np.nan

def eN3PointOED(Z,N,W,df):
    try:
        return ceil((eOneNSE(Z,N,W,df) + eOneNSE(Z,N+1,W,df))/2)
    except:
        return np.nan

def eP3PointOED(Z,N,W,df):
    try:
        return ceil((eOnePSE(Z,N,W,df) + eOnePSE(Z+1,N,W,df))/2)
    except:
        return np.nan

def eSNESplitting(Z,N,W,df):
    try:
        return ceil((eOneNSE(Z,N,W,df) + eOneNSE(Z,N+2,W,df))/2)
    except:
        return np.nan

def eSPESplitting(Z,N,W,df):
    try:
        return ceil((eOnePSE(Z,N,W,df) + eOnePSE(Z+2,N,W,df))/2)
    except:
        return np.nan

def eWignerEC(Z,N,W,df):
    if isinstance(Z, (np.ndarray, pd.Series)):
        WECarr = []
        for z, n in zip(Z, N):
            try:
                WECarr.append(ceil((eDoubleMDiff(z,n,W,df)+eDoubleMDiff(z-2,n,W,df) + eDoubleMDiff(z,n+2,W,df))/3))
            except:
                WECarr.append(np.nan)
        return np.where(Z == N, WECarr, np.nan)
    elif Z!=N:
        return np.nan
    return ceil((eDoubleMDiff(Z,N,W,df)+eDoubleMDiff(Z-2,N,W,df)+eDoubleMDiff(Z,N+2,W,df))/3)

def eBEperA(Z,N,W,df):
    try:
        return (eBE(Z,N,W,df))
    except:
        return np.nan
    
def Wig1(Z, N):
    return (1.8*np.exp(-380*((N-Z)/(N+Z))**2))-(.84*abs(N-Z)*np.exp(-(((N+Z)/26)**2)))

def Wig2(Z, N):
    return -47*(abs(N-Z)/(N+Z))

def Identity(Z, N):
    return 0